In [2]:
import pandas as pd
import re
from packaging import version
import operator
import matplotlib
import matplotlib.pyplot as plt

In [3]:
lts_list = ['0-7', '2-22', '3-22', '6-35', '7-24', '9-21', '11-22', '12-14', '12-26', '13-11', '13-19', '14-27', '15-3', '16-11']
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../../data/test/lts-{lts}/lts-{lts}.df')
    df_list.append(df)

In [4]:
df_list[0][df_list[0]['package']=='QuickCheck']

,package,version,stability,cabal-file,categories,deps,provided-modules,src-dirs,main-modules,mtl-direct,version-range-deps
0,abstract-deque,0.3,[],C:/Users/nicol/Desktop/lts/lts-0-7\abstract-de...,[data],"[array, base, containers, random, time]","[Data.Concurrent.Deque.Class, Data.Concurrent....",[],[],0,"[(array, -any), (base, ==4.*), (containers, -a..."
1,abstract-par,0.3.3,[experimental],C:/Users/nicol/Desktop/lts/lts-0-7\abstract-pa...,"[control, parallelism, monads]","[base, deepseq]","[Control.Monad.Par.Class, Control.Monad.Par.Un...",[],[],0,"[(base, ==4.*), (deepseq, >=1.1)]"
2,AC-Vector,2.3.2,[experimental],C:/Users/nicol/Desktop/lts/lts-0-7\AC-Vector\A...,"[data, math, numerical, graphics]",[base],"[Data.BoundingBox.B1, Data.BoundingBox.B2, Dat...",[.],[],0,"[(base, ==4.*)]"
3,accelerate,0.15.0.0,[experimental],C:/Users/nicol/Desktop/lts/lts-0-7\accelerate\...,"[compilers/interpreters, concurrency, data, pa...","[array, base, containers, fclabels, ghc-prim, ...","[Data.Array.Accelerate, Data.Array.Accelerate....",[],[],0,"[(array, >=0.3), (base, ==4.7.*), (containers,..."
4,ace,0.6,[],C:/Users/nicol/Desktop/lts/lts-0-7\ace\ace-0.6...,[linguistics],"[attoparsec, base, blaze-html, blaze-markup, d...","[ACE, ACE.Combinators, ACE.Html, ACE.Parsers, ...",[src/],[],0,"[(attoparsec, >=0.10.4.0), (base, ==4.*), (bla..."


In [62]:
for idx,df in enumerate(df_list):
    print('-----------------------')
    print('Processing LTS-',lts_list[idx])
    foo(df)

In [63]:
out_range = []
out_range_percentage =[]
total = []
list_out_range = []
list_in_range = []
list_any = []
for idx,df in enumerate(df_list):
    dict_out_range = []
    dict_in_range = []
    dict_any = []
    count_out_range = 0
    count_total = 0
    for idx,row in df.iterrows():
        pkg = row['package']
        deps = row['dependencies_status']
        for dep in deps:
            if deps[dep] == 'OUT_RANGE':
                count_out_range+=1
                dict_out_range.append((pkg,dep))
            elif deps[dep] == 'IN_RANGE':
                dict_in_range.append(dep)
            else:
                dict_any.append(dep)
            count_total+=1
    list_out_range.append(dict_out_range)
    list_in_range.append(dict_in_range)
    list_any.append(dict_any)
    out_range.append(count_out_range)
    out_range_percentage.append(count_out_range/count_total)
    total.append(count_total)

{'hola': [[5, 5, 'hola', 'uwu'], [5, 6, 'hola', 'uwu']]}

In [78]:
list_conflict = []
percentage = []
unique_out_range = []
for idx,array in enumerate(list_out_range):
    count = 0
    pkg_out_range = []
    print('lts-',lts_list[idx])
    for pkg,dep in list_out_range[idx]:
        if dep in list_in_range[idx]:
            count+=1
        else:
            pkg_out_range.append((pkg,dep))
    pkg_out_range = pkg_out_range
    unique_out_range.append(pkg_out_range)
    list_conflict.append(count)
    percentage.append(count/total[idx])
    print('Total dependencies with OUT_RANGE: ',len(list_out_range[idx]))
    print('Total dependencies with IN_RANGE: ',len(list_in_range[idx]))
    print('Total dependencies with ANY: ',len(list_any[idx]))
    print('Total dependencies: ',total[idx])

['4', '7', '0', '1'] ['2']
['0', '5', '0', '0'] ['0', '3']
['4', '7', '0', '1'] ['4', '0']
['4', '2'] ['1', '2']
['0', '16', '0', '1'] ['0', '1']
['0', '5', '0', '0'] ['0', '2']
['4', '7', '0', '1'] ['4', '5']
['0', '5', '5', '1'] ['0', '2']
['4', '9'] ['4', '6', '1']


IndexError: list index out of range

In [69]:
df = df_list[0]
pkg_to_analise = 'aeson'
for idx,row in df.iterrows():
    pkg = row['package']
    deps = row['deps']
    range = row['dependencies_status']
    for dep in deps:
        if dep == pkg_to_analise:
            print('pkg:',pkg,'deps: ',dep,'status: ',range[dep])

,package,version,stability,cabal-file,categories,deps,provided-modules,src-dirs,main-modules,mtl-direct,version-range-deps
0,abstract-deque,0.3,[],C:/Users/nicol/Desktop/lts/lts-6-35\abstract-d...,[data],"[array, base, containers, random, time]","[Data.Concurrent.Deque.Class, Data.Concurrent....",[],[],0,"[(array, -any), (base, ==4.*), (containers, -a..."


In [75]:
fig, ax = plt.subplots(figsize=(12, 10))
ax.margins(0.015) 
plt.subplots_adjust(right=1.1)
print(out_range)
bar = ax.bar(x=lts_list, height=out_range_percentage, width=0.8, color='darkgrey', align='center', label='Dependdencies out of range')
ax.bar_label(bar, labels=list(out_range), label_type='center', color='black', fontsize=15)

ax.set_xticks(lts_list)
ax.set_xticklabels(lts_list,fontsize=15)
ax.set_yticks([0.005, 0.010, 0.015,0.020])
ax.set_yticklabels(['0.005%', '0.010%', '0.015%','0.020%'],fontsize=15)
ax.set_xlabel('LTS release',fontsize=18)
ax.set_ylabel('Porcentage of dependency versions out of range',fontsize=18)

plt.savefig("../../graphs/rq1_2.pdf", format='pdf', bbox_inches='tight')
plt.show()

uwu


In [77]:
fig, ax = plt.subplots(figsize=(12, 10))
ax.margins(0.015) 
plt.subplots_adjust(right=1.1)

bar = ax.bar(x=lts_list, height=percentage, width=0.8, color='darkgrey', align='center', label='Dependdencies inconsistent')
ax.bar_label(bar, labels=list(list_conflict), label_type='center', color='black', fontsize=15)


----------------------------------------
Package ace
Name_dep attoparsec
version_dep 0.12.1.2
version_pkg >=0.10.4.0
State atrasada
----------------------------------------
----------------------------------------
Package ace
Name_dep blaze-html
version_dep 0.7.0.3
version_pkg >=0.5.1.0
State atrasada
----------------------------------------
----------------------------------------
Package ace
Name_dep text
version_dep 1.1.1.3
version_pkg >=0.11.2.0
State atrasada
----------------------------------------
----------------------------------------
Package active
Name_dep array
version_dep 0.5.0.0
version_pkg >=0.3 && <0.6
State adelantada
----------------------------------------
----------------------------------------
Package active
Name_dep base
version_dep 4.7.0.1
version_pkg >=4.0 && <4.8
State adelantada
----------------------------------------
----------------------------------------
Package active
Name_dep semigroupoids
version_dep 4.2
version_pkg >=1.2 && <4.3
State adelantada
---